In [30]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import datetime as dt
from dateutil.parser import parse


In [32]:
# extract just the chiller features of interest from the dataframe
# CH_features = list of features
# CH = chiller number
# df = dataframe with all the data
def chiller_data(df, CH, CH_features):
    df_chiller = pd.DataFrame()
    df_chiller['datetime'] = df['timestamp'].apply(get_datetime)
    
    for f in CH_features:
        if f[0] == '_':
            ff = f[1:]
        else:
            ff = f
        feature = 'CH'+CH+f
        df_chiller[ff] = df[feature]
        
    return df_chiller

def get_datetime(y):
    x = parse(y)
    return x

In [33]:
CH_features = ['_kW/Ton','Load','CDWRT','CDWST','DTLift','Ton','_EvapApproach',
               '_CondApproach','Hz','_CompSH','IGV','REFLVL']

## Samford Data

Chillers 1-4 are **1500 Ton** variable speed YK chillers   

In [34]:
df = pd.read_csv('samford_chiller/dss_Samford_chillers.csv')

In [102]:
df.columns[4:53]

Index(['CH1_CDW_DT', 'CH1_CHLRcmd_Alarm', 'CH1_CHW_DT', 'CH1_CHWSTSP_Alarm',
       'CH1_CompSH', 'CH1_CondApproach', 'CH1_DP_Lift', 'CH1_DT_Lift',
       'CH1_EvapApproach', 'CH1_kW/Ton', 'CH1ALARM', 'CH1AVAIL', 'CH1CDP',
       'CH1CDT', 'CH1CDWRT', 'CH1CDWST', 'CH1CDWVLV', 'CH1CHWDP', 'CH1CHWFLO',
       'CH1CHWFLO_Meter', 'CH1CHWRT', 'CH1CHWST', 'CH1CHWSTSP', 'CH1CHWVLV',
       'CH1COMDISCHT', 'CH1CondApproach', 'CH1CYCLF', 'CH1DMD', 'CH1DPLift',
       'CH1DTLift', 'CH1EvapApproach', 'CH1EVP', 'CH1EVT', 'CH1F', 'CH1FLA',
       'CH1Hz', 'CH1IGV', 'CH1kW', 'CH1Lift', 'CH1Load', 'CH1REFLVL', 'CH1S',
       'CH1SBoolean', 'CH1SPD', 'CH1SS', 'CH1STATE', 'CH1SURGECNT', 'CH1Ton',
       'CH1WARNF'],
      dtype='object')

In [29]:
df.columns


Index(['timestamp', 'CH3LoadDisp', 'CH3PctLoad', 'CH4LoadDisp', 'CH4PctLoad',
       'CH5LoadDisp', 'CH5PctLoad', 'CHkW%', 'CHTON%', 'PercentAnnualSavings',
       ...
       'CH5SURGE', 'LOOPREQ', 'RUNCHLR3', 'RUNCHLR4', 'RUNCHLR5',
       'CH3SURGECNT', 'CH4SURGECNT', 'CH5SURGECNT', 'CLGMODE', 'StationSTG'],
      dtype='object', length=414)

In [39]:
# Samford Data

chillers = ['1','2','3','4']

df_chiller = chiller_data(df,'1',CH_features)
for c in chillers:
    df_chiller = chiller_data(df,c,CH_features)
    df_chiller.drop(df_chiller[df_chiller.Load == 0].index, inplace=True)
    df_chiller.drop(df_chiller[df_chiller['kW/Ton'] == 0].index, inplace=True)
    df_chiller.to_csv('data/S'+c+'_chiller.csv')

In [40]:
df_chiller.head()

,datetime,kW/Ton,Load,CDWRT,CDWST,DTLift,Ton,EvapApproach,CondApproach,Hz,CompSH,IGV,REFLVL
3064,2018-01-11 15:20:00+00:00,0.341737,0.489655,76.400002,69.500000,32.500000,734.482326,0.400002,0.199997,44.740002,9.500000,94.260002,15.700000
3065,2018-01-11 15:25:00+00:00,0.318592,0.324344,74.000000,69.099998,27.500000,486.516164,0.099998,-0.099998,43.349998,11.299995,3.930000,20.299999
3066,2018-01-11 15:30:00+00:00,0.356950,0.448242,75.300003,68.500000,30.900002,672.362777,0.400002,0.199997,45.740002,10.400002,14.240000,26.200001
3067,2018-01-11 15:35:00+00:00,0.349066,0.467915,75.400002,68.400002,30.900002,701.872192,0.500000,0.199997,45.340000,9.900002,20.629999,26.000000
3068,2018-01-11 15:40:00+00:00,0.353166,0.487023,75.800003,68.599998,31.400002,730.534315,0.500000,0.099998,45.139999,9.699997,32.029999,26.400000


## Albuquerque Data

CH3 YKCSCSQ5-CJG **350 Ton** Chiller
CH4   
CH5   

In [5]:
df = pd.read_csv('Albuquerque/dss_Alb h 2017-11.csv')

In [28]:
print(df.columns[0:3])
print(df.columns[27:47])

Index(['timestamp', 'CH3LoadDisp', 'CH3PctLoad'], dtype='object')
Index(['CH3CDPST', 'CH3CDT', 'CH3CDTSC', 'CH3CDW_DeltaT', 'CH3CDWDT',
       'CH3CDWRT', 'CH3CDWST', 'CH3CHW_DeltaT', 'CH3CHWDT', 'CH3CHWRT',
       'CH3CHWST', 'CH3CHWSTSP', 'CH3CompDiscSuperheat', 'CH3CondApproach',
       'CH3DISTEMP', 'CH3DTLift', 'CH3EvapApproach', 'CH3EVPST', 'CH3EVT',
       'CH3Lift'],
      dtype='object')


LoadDisp =    
PctLoad  = Percent Load   
CDPST    =    
CDT      = Condenser Refrigerant Temperature   
CDTSC    =    
CHW_DeltaT = 
CHWDT    = Chilled Water Delta T  
CHWRT    = Chilled Water Return Temperature   
CHWST    = Chilled Water Supply Temperature   
CHWSTSP  =   
CompDiscSuperheat = 
CondApproach = 
DISTEMP  = Compressor Refrigerant Discharge Temp  
DTLift   = Lift
EvapApproach = 
EVPST    = Evaporator Supply Temperature  
EVT      = Evaporator Refrigerant Temperature  
Lift     = 

In [20]:
CH_features = ['_kW/Ton','PctLoad','CDWRT','CDWST','DTLift','Ton','EvapApproach',
               'CondApproach','Hz','CompDiscSuperheat','IGV','REFLVL']

In [21]:
df_chiller = chiller_data(df,'3',CH_features)
df_chiller.to_csv('data/A3_chiller.csv')

KeyError: 'CH3Hz'